In [4]:
%pip install scholarly
%pip install requests beautifulsoup4

     -------------------------------------- 51.9/51.9 kB 886.0 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 50.5/50.5 kB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 75.4/75.4 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 2.8/2.8 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 6.6/6.6 MB 4.9 MB/s eta 0:00:00
     ---------------------------------------- 66.4/66.4 kB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 3.8/3.8 MB 9.6 MB/s eta 0:00:00
     ---------------------------------------- 72.5/72.5 kB ? eta 0:00:00
     ------------------------------------- 384.9/384.9 kB 11.7 MB/s eta 0:00:00
     -------------------------------------- 570.0/570.0 kB 8.9 MB/s eta 0:00:00
     ----

In [6]:
from scholarly import scholarly, MaxTriesExceededException, ProxyGenerator
import pickle
import os
import time, random
import math
import requests
from bs4 import BeautifulSoup

In [4]:
class RelatedWork:
    def __init__(self):
        # set free proxy
        #pg = ProxyGenerator()
        #pg.FreeProxies()
        #scholarly.use_proxy(pg)
        
        self.paper_list = {}
        self.is_updated = False
        self.load_paper_list()
        
    def build_paper_list_from_txt(self):
        if os.path.exists('./paper_list.txt'): 
            f = open('paper_list.txt', 'r')
            lines = f.readlines()
            for i, line in enumerate(lines):
                items = line.split('_')
                year = items[0]
                title = items[1].split('\n')[0]
                self.paper_list[i+1] = {'title': title, 'year': year}
            self.is_updated = True
        else: # txt 파일이 없으면 빈 딕셔너리 반환
            print('No txt file')
            
        # Save data_dict
        self.update_pkl()
          
    def load_pkl(self):
        f =  open('paper_list.pkl','rb')
        self.paper_list = pickle.load(f)
    
    def update_pkl(self):
        if self.is_updated: # 업데이트 사항이 있는 경우에만
            print('update pkl')
            with open('paper_list.pkl','wb') as f:
               pickle.dump(self.paper_list, f)
            self.is_updated = False
        else:
            print('no update')
    
    def load_paper_list(self):
        if os.path.exists('./paper_list.pkl'): # pkl 파일이 존재하면 불러옴
            print('load from pkl')
            self.load_pkl()
        else: # pkl 파일이 없으면 
            print('load from txt')
            self.build_paper_list_from_txt()
    
    def update_paper_info_from_scholar(self, paper_num):
        paper = self.paper_list[paper_num]
        if 'info' not in paper.keys(): # info 정보가 없는 경우에만 업데이트
            title = paper['title']
            while True:
                try:
                    search_query = scholarly.search_pubs(title)
                    break
                except MaxTriesExceededException as err:
                    print("Google Scholar is aggressively blocking us! Quitting for now.")
                    self.updateFreeProxy()
            paper_info = next(search_query)
            self.paper_list[paper_num]['info'] = paper_info
            print(title, ' information updated')
            self.is_updated = True
        else:
            print('info already exists')
        
    def update_all_info(self):
        for key in self.paper_list.keys():
            time.sleep(random.uniform(2, 5))
            self.update_paper_info_from_scholar(key)
            self.update_pkl()
            print('paper {} updated\n'.format(key))

    def update_citedby(self, paper_num):
        paper = self.paper_list[paper_num]
        print(paper['title'])
        if 'citedby' not in paper.keys(): # citedby 정보가 없는 경우에만 업데이트
            while True:
                try:
                    citation_list = scholarly.citedby(paper['info'])
                    cl = []
                    for i, c in enumerate(citation_list):
                        print(c['bib']['title'])
                        cl.append(c['bib'])
                    paper['citedby'] = cl            
                    self.is_updated = True
                    break
                except MaxTriesExceededException as err:
                    print("Google Scholar is aggressively blocking us! Quitting for now.")
                    self.updateFreeProxy()
                except Exception as e: 
                    print(e)
                    if e == 'citedby_url': # cited by no paper
                        paper['citedby'] = []
                    break 
        else:
            print('citedby already exists')
            
    def update_all_citedby(self):
        for key in self.paper_list.keys():
            #time.sleep(random.uniform(2, 5))
            self.update_citedby(key)
            self.update_pkl()
            print('paper {} updated\n'.format(key))
            
    def updateFreeProxy(self):
        pg = ProxyGenerator()
        pg.FreeProxies()
        print("update FreeProxy!")
        scholarly.use_proxy(pg)

In [5]:
rw = RelatedWork()

load from pkl


In [7]:
rw.update_all_citedby()

A Task-Agnostic Regularizer for Diverse Subpolicy Discovery in Hierarchical Reinforcement Learning
citedby already exists
no update
paper 1 updated

Hierarchical Kickstarting for Skill Transfer in Reinforcement Learning
citedby already exists
no update
paper 2 updated

Multi-task Hierarchical Adversarial Inverse Reinforcement Learning
'citedby_url'
no update
paper 3 updated

Accelerating Reinforcement Learning for Autonomous Driving using Task-Agnostic and Ego-Centric Motion Skills
citedby already exists
no update
paper 4 updated

Advanced Skills through Multiple Adversarial Motion Priors in Reinforcement Learning
citedby already exists
no update
paper 5 updated

ASPiRe: Adaptive Skill Priors for Reinforcement Learning
'citedby_url'
no update
paper 6 updated

Cascaded Compositional Residual Learning for Complex Interactive Behaviors
citedby already exists
no update
paper 7 updated

CEIP: Combining Explicit and Implicit Priors for Reinforcement Learning with Demonstrations
'citedby_url'

KeyboardInterrupt: 

In [4]:
rw.paper_list[15]['info']

{'container_type': 'Publication',
 'source': <PublicationSource.PUBLICATION_SEARCH_SNIPPET: 'PUBLICATION_SEARCH_SNIPPET'>,
 'bib': {'title': 'Skill-based meta-reinforcement learning',
  'author': ['T Nam', 'SH Sun', 'K Pertsch', 'SJ Hwang'],
  'pub_year': '2022',
  'venue': 'arXiv preprint arXiv …',
  'abstract': 'While deep reinforcement learning methods have shown impressive results in robot learning, their sample inefficiency makes the learning of complex, long-horizon behaviors with real robot systems infeasible. To mitigate this issue, meta-reinforcement learning methods aim to enable fast learning on novel tasks by learning how to learn. Yet, the application has been limited to short-horizon tasks with dense rewards. To enable learning long-horizon behaviors, recent works have explored leveraging prior experience in the form'},
 'filled': False,
 'gsrank': 1,
 'pub_url': 'https://arxiv.org/abs/2204.11828',
 'author_id': ['', 'uXsfnaQAAAAJ', '3oe0I0QAAAAJ', 'RP4Qx3QAAAAJ'],
 'url_

In [5]:
citation_list = scholarly.citedby(rw.paper_list[15]['info'])

In [6]:
rw.update_citedby(15)

Skill-based Meta-Reinforcement Learning
1
2
Towards continual reinforcement learning: A review and perspectives
How to reuse and compose knowledge for a lifetime of tasks: A survey on continual learning and functional composition
Hyper-decision transformer for efficient online policy adaptation
Learning Options via Compression
Efficient Multi-Task Reinforcement Learning via Selective Behavior Sharing
Guided Skill Learning and Abstraction for Long-Horizon Manipulation
Meta-Learning Transferable Parameterized Skills
A Survey of Meta-Reinforcement Learning
CEIP: Combining Explicit and Implicit Priors for Reinforcement Learning with Demonstrations
Uncertainty-based Meta-Reinforcement Learning for Robust Radar Tracking
Skills Regularized Task Decomposition for Multi-task Offline Reinforcement Learning
SMPL: Simulated Industrial Manufacturing and Process Control Learning Environments
Meta-Reinforcement Learning via Exploratory Task Clustering
The StarCraft Multi-Agent Challenges+: Learning o

In [25]:
print(rw.paper_list)

{1: {'title': 'A Task-Agnostic Regularizer for Diverse Subpolicy Discovery in Hierarchical Reinforcement Learning', 'year': '2023', 'info': {'container_type': 'Publication', 'source': <PublicationSource.PUBLICATION_SEARCH_SNIPPET: 'PUBLICATION_SEARCH_SNIPPET'>, 'bib': {'title': 'A task-agnostic regularizer for diverse subpolicy discovery in hierarchical reinforcement learning', 'author': 'Huo, Liangyu and Wang, Zulin and Xu, Mai and Song, Yuhang', 'pub_year': '2022', 'venue': 'IEEE Transactions on Systems …', 'abstract': 'The automatic subpolicy discovery approach in hierarchical reinforcement learning (HRL) has recently achieved promising performance on sparse reward tasks. This accelerates transfer learning and unsupervised intelligent creatures while eliminating the domain-specific knowledge constraint. Most previously developed approaches are demonstrated to suffer from collapsing into the situation where one subpolicy dominates the whole task, since they cannot ensure the diversit

In [11]:
rw.update_paper_info_from_scholar(1)

info already exists


In [12]:
rw.update_all_info()

info already exists
no update
paper 1 updated

info already exists
no update
paper 2 updated

info already exists
no update
paper 3 updated

info already exists
no update
paper 4 updated

info already exists
no update
paper 5 updated

info already exists
no update
paper 6 updated

info already exists
no update
paper 7 updated

info already exists
no update
paper 8 updated

info already exists
no update
paper 9 updated

info already exists
no update
paper 10 updated

info already exists
no update
paper 11 updated

info already exists
no update
paper 12 updated

Residual Skill Policies: Learning an Adaptable Skill-based Action Space for Reinforcement Learning for Robotics  information updated
update pkl
paper 13 updated

Robot Learning of Mobile Manipulation With Reachability Behavior Priors  information updated
update pkl
paper 14 updated

Skill-based Meta-Reinforcement Learning  information updated
update pkl
paper 15 updated

Skill-based Model-based Reinforcement Learning  information

In [13]:
rw.paper_list[1]['info']['pub_url']

'https://ieeexplore.ieee.org/abstract/document/9906972/'

In [14]:
rw.paper_list[3]['info']

{'container_type': 'Publication',
 'source': <PublicationSource.PUBLICATION_SEARCH_SNIPPET: 'PUBLICATION_SEARCH_SNIPPET'>,
 'bib': {'title': 'Multi-task Hierarchical Adversarial Inverse Reinforcement Learning',
  'author': ['J Chen', 'D Tamboli', 'T Lan', 'V Aggarwal'],
  'pub_year': '2023',
  'venue': 'arXiv preprint arXiv:2305.12633',
  'abstract': 'Multi-task Imitation Learning (MIL) aims to train a policy capable of performing a distribution of tasks based on multi-task expert demonstrations, which is essential for general-purpose robots. Existing MIL algorithms suffer from low data efficiency and poor performance on complex long-horizontal tasks. We develop Multi-task Hierarchical Adversarial Inverse Reinforcement Learning (MH-AIRL) to learn hierarchically-structured multi-task policies, which is more beneficial for compositional tasks with long horizons and has higher expert'},
 'filled': False,
 'gsrank': 1,
 'pub_url': 'https://arxiv.org/abs/2305.12633',
 'author_id': ['k0KJm7k

In [15]:
scholar_url = 'https://scholar.google.co.kr/'
citedby_url = scholar_url + rw.paper_list[1]['info']['citedby_url']
print(citedby_url)

https://scholar.google.co.kr//scholar?cites=523997480481690644&as_sdt=5,33&sciodt=0,33&hl=en


In [4]:
rw.update_citedby(2)

{'title': 'Hierarchical Kickstarting for Skill Transfer in Reinforcement Learning', 'year': '2023', 'info': {'container_type': 'Publication', 'source': <PublicationSource.PUBLICATION_SEARCH_SNIPPET: 'PUBLICATION_SEARCH_SNIPPET'>, 'bib': {'title': 'Hierarchical kickstarting for skill transfer in reinforcement learning', 'author': ['M Matthews', 'M Samvelyan', 'J Parker-Holder'], 'pub_year': '2022', 'venue': 'arXiv preprint arXiv …', 'abstract': 'Practising and honing skills forms a fundamental component of how humans learn, yet artificial agents are rarely specifically trained to perform them. Instead, they are usually trained end-to-end, with the hope being that useful skills will be implicitly learned in order to maximise discounted return of some extrinsic reward function. In this paper, we investigate how skills can be incorporated into the training of reinforcement learning (RL) agents in complex environments with large state-action spaces and sparse rewards. To this end, we'}, 'fi

In [26]:
rw.paper_list[42]['citedby']

[{'title': 'A review of robot learning for manipulation: Challenges, representations, and algorithms',
  'author': ['O Kroemer', 'S Niekum', 'G Konidaris'],
  'pub_year': '2021',
  'venue': 'The Journal of Machine Learning …',
  'abstract': 'A key challenge in intelligent robotics is creating robots that are capable of directly interacting with the world around them to achieve their goals. The last decade has seen'},
 {'title': "Solving rubik's cube with a robot hand",
  'author': ['I Akkaya', 'M Andrychowicz', 'M Chociej', 'M Litwin'],
  'pub_year': '2019',
  'venue': 'arXiv preprint arXiv …',
  'abstract': 'We demonstrate that models trained only in simulation can be used to solve a manipulation problem of unprecedented complexity on a real robot. This is made possible by two key'},
 {'title': 'Play, curiosity, and cognition',
  'author': ['J Chu', 'LE Schulz'],
  'pub_year': '2020',
  'venue': 'Annual Review of Developmental …',
  'abstract': 'Few phenomena in childhood are as compe

In [9]:
rw.update_all_citedby()

A Task-Agnostic Regularizer for Diverse Subpolicy Discovery in Hierarchical Reinforcement Learning
citedby already exists
no update
paper 1 updated

Hierarchical Kickstarting for Skill Transfer in Reinforcement Learning
Reincarnating reinforcement learning: Reusing prior computation to accelerate progress
Hierarchical Learning with Unsupervised Skill Discovery for Highway Merging Applications
update pkl
paper 2 updated

Multi-task Hierarchical Adversarial Inverse Reinforcement Learning


KeyError: 'citedby_url'

In [9]:
paper_list = {}
f = open('paper_list.txt', 'r')
lines = f.readlines()

for i, line in enumerate(lines):
    items = line.split('_')
    year = items[0]
    title = items[1].split('\n')[0]
    #print(i+1, year, title)
    paper_list[i+1] = {'title': title, 'year': year}
    #print(paper_list[i+1])
    #search_query = scholarly.search_pubs(title)
    #first_author_result = next(search_query)
    #print(first_author_result)
    #citation_list = scholarly.citedby(first_author_result)
    #cl = []
    #for c in citation_list:
    #    title = c['bib']['title']
    #    year = c['bib']['pub_year']
    #    cl.append({'title': title, 'year': year})
    #paper_list[i+1]['cited_by'] = cl

In [54]:
search_query = scholarly.search_pubs('Learning Latent Plans from Play')
first_author_result = next(search_query)
print(first_author_result)

{'container_type': 'Publication',
 'source': <PublicationSource.PUBLICATION_SEARCH_SNIPPET: 'PUBLICATION_SEARCH_SNIPPET'>,
 'bib': {'title': 'Bayesian controller fusion: Leveraging control priors in deep reinforcement learning for robotics',
  'author': ['K Rana', 'V Dasagi', 'J Haviland', 'B Talbot'],
  'pub_year': '2021',
  'venue': 'arXiv preprint arXiv …',
  'abstract': 'We present Bayesian Controller Fusion (BCF): a hybrid control strategy that combines the strengths of traditional hand-crafted controllers and model-free deep reinforcement learning (RL). BCF thrives in the robotics domain, where reliable but suboptimal control priors exist for many tasks, but RL from scratch remains unsafe and data-inefficient. By fusing uncertainty-aware distributional outputs from each system, BCF arbitrates control between them, exploiting their respective strengths. We study BCF on two real-world robotics tasks'},
 'filled': False,
 'gsrank': 1,
 'pub_url': 'https://arxiv.org/abs/2107.09822',


In [62]:
citation_list = scholarly.citedby(first_author_result)

In [63]:
cl = []
for i, c in enumerate(citation_list):
    cl.append(c)

In [92]:
print('cited by')
for c in cl:
    title = c['bib']['title']
    year = c['bib']['pub_year']
    #print(title)
    for key in paper_list.keys():
        if paper_list[key]['title'].lower() == title.lower():
            print(year, title)

cited by
2023 Residual skill policies: Learning an adaptable skill-based action space for reinforcement learning for robotics
2022 Robot learning of mobile manipulation with reachability behavior priors


In [ ]:
# 관련 논문 딕셔너리 작성
# 각 논문별 어떤 논문으로부터 인용되었는지 찾기 -> 딕셔너리에 저장
# 마인드맵에 반영
# 핵심 논문 선정 -> 내용 파악

In [3]:
rw = RelatedWork()

load from pkl


In [16]:
for key in rw.paper_list.keys():
    paper = rw.paper_list[key]
    if 'citedby' not in paper:
         print(key, paper['title'])

3 Multi-task Hierarchical Adversarial Inverse Reinforcement Learning
6 ASPiRe: Adaptive Skill Priors for Reinforcement Learning
8 CEIP: Combining Explicit and Implicit Priors for Reinforcement Learning with Demonstrations
35 Learning quadrupedal locomotion over challenging terrain
37 Multi-expert learning of adaptive legged locomotion
38 Multiplicative controller fusion: Leveraging algorithmic priors for sample-efficient reinforcement learning and safe sim-to-real transfer
39 PARROT: DATA-DRIVEN BEHAVIORAL PRIORS FOR REINFORCEMENT LEARNING
40 Directed-info GAIL: Learning hierarchical policies from unsegmented demonstrations using directed information
41 Dynamics-Aware Unsupervised Discovery of Skills
43 LEARNING MULTI-LEVEL HIERARCHIES WITH HINDSIGHT
44 MCP: Learning Composable Hierarchical Control with Multiplicative Compositional Policies
45 Neural probabilistic motor primitives for humanoid control
46 Relay Policy Learning: Solving Long-Horizon Tasks via Imitation and Reinforcement 

# Load citedby list from html

### Generate instance

In [4]:
rw = RelatedWork()

load from pkl


### Update citedby page list

In [5]:
for key in rw.paper_list.keys():
    paper = rw.paper_list[key]
    # maximum # of citedby page
    max_page = math.ceil(paper['info']['num_citations']/10.0)
    print('max_page: ', max_page)

    # load citedby page url list
    page_list = []

    scholar_url = 'https://scholar.google.co.kr'
    try:
        first_page = scholar_url + paper['info']['citedby_url']
        page_list.append(first_page)
        print(first_page)

        paper_id = first_page.split('cites=')[1].split('&')[0]

        for page in range(max_page-1):
            start = (page+1)*10
            next_page = "https://scholar.google.co.kr/scholar?start={}&hl=en&as_sdt=5,33&sciodt=0,33&cites={}&scipsc=".format(start, paper_id)
            page_list.append(next_page)
            print(next_page)
    except KeyError as e:
        print('No citation')
        
    # save page list
    rw.paper_list[key]['citedby_page_list'] = page_list
    if 'citedby_paper_list' in paper:
        rw.paper_list[key]['citedby_paper_num'] = len(paper['citedby_paper_list'])
    else:
        rw.paper_list[key]['citedby_paper_num'] = 0

    # update pickle
    rw.is_updated = True
    rw.update_pkl()

max_page:  1
https://scholar.google.co.kr/scholar?cites=523997480481690644&as_sdt=5,33&sciodt=0,33&hl=en
update pkl
max_page:  1
https://scholar.google.co.kr/scholar?cites=18041795639441247667&as_sdt=5,33&sciodt=0,33&hl=en&oe=ASCII
update pkl
max_page:  0
No citation
update pkl
max_page:  1
https://scholar.google.co.kr/scholar?cites=4828484264646918391&as_sdt=5,33&sciodt=0,33&hl=en
update pkl
max_page:  2
https://scholar.google.co.kr/scholar?cites=5877439646516921010&as_sdt=5,33&sciodt=0,33&hl=en
https://scholar.google.co.kr/scholar?start=10&hl=en&as_sdt=5,33&sciodt=0,33&cites=5877439646516921010&scipsc=
update pkl
max_page:  0
No citation
update pkl
max_page:  1
https://scholar.google.co.kr/scholar?cites=12891288254842573229&as_sdt=5,33&sciodt=0,33&hl=en
update pkl
max_page:  0
No citation
update pkl
max_page:  1
https://scholar.google.co.kr/scholar?cites=9163542561991441433&as_sdt=5,33&sciodt=0,33&hl=en
update pkl
max_page:  1
https://scholar.google.co.kr/scholar?cites=88896927617406

### Load citedby papers from citedby pages

In [6]:
def get_papers_from_citedby_page(page_url):
    response = requests.get(page_url)
    print(response)

    if response.status_code == 200:
        # Parse HTML with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        #print(soup)
        body = soup.select('h3.gs_rt')
        print(body)
        paper_list = []
        if len(body) != 0:
            for i, b in enumerate(body):
                paper = {}
                title = b.text.split('] ')[-1]
                print(title)
                paper['title'] = title
                try:
                    paper['url'] = b.select('a')[0]['href']
                except IndexError as e:
                    print('No url')
                paper_list.append(paper)
            return paper_list
        else:
            print('failed to load citedby page')
            return None
    else:
        print('error')
        return None

In [36]:
for key in rw.paper_list.keys():
    paper = rw.paper_list[key]
    print('\n', '[', key, ']', paper['title'])
    print(paper['info']['num_citations'], paper['citedby_paper_num'])
    if paper['citedby_paper_num'] < paper['info']['num_citations']: # 개수가 적으면
        citedby_paper_list = []
        if 'citedby_paper_list' not in rw.paper_list[key]:
            rw.paper_list[key]['citedby_paper_list'] = [] 
        for i, page in enumerate(paper['citedby_page_list']):
            print(page)
            if (i+1)*10 <= paper['citedby_paper_num']: # 채운곳까지는 넘어감
                print('continue {} < {}'.format((i+1)*10, paper['citedby_paper_num']))
                continue 
            #time.sleep(2)
            return_list = get_papers_from_citedby_page(page)
            if return_list is not None:
                citedby_paper_list += return_list
            else:
                break
        print(citedby_paper_list)
        rw.paper_list[key]['citedby_paper_list'] += citedby_paper_list
        rw.paper_list[key]['citedby_paper_num'] += len(citedby_paper_list)
        print(rw.paper_list[key]['citedby_paper_num'])
        rw.is_updated = True
        rw.update_pkl()
    else:
        print('already exists')
    


 [ 1 ] A Task-Agnostic Regularizer for Diverse Subpolicy Discovery in Hierarchical Reinforcement Learning
2 2
already exists

 [ 2 ] Hierarchical Kickstarting for Skill Transfer in Reinforcement Learning
2 2
already exists

 [ 3 ] Multi-task Hierarchical Adversarial Inverse Reinforcement Learning
0 0
already exists

 [ 4 ] Accelerating Reinforcement Learning for Autonomous Driving using Task-Agnostic and Ego-Centric Motion Skills
1 1
already exists

 [ 5 ] Advanced Skills through Multiple Adversarial Motion Priors in Reinforcement Learning
16 16
already exists

 [ 6 ] ASPiRe: Adaptive Skill Priors for Reinforcement Learning
0 0
already exists

 [ 7 ] Cascaded Compositional Residual Learning for Complex Interactive Behaviors
1 1
already exists

 [ 8 ] CEIP: Combining Explicit and Implicit Priors for Reinforcement Learning with Demonstrations
0 0
already exists

 [ 9 ] MPR-RL: Multi-Prior Regularized Reinforcement Learning for Knowledge Transfer
5 5
already exists

 [ 10 ] Planning to P

In [23]:
# 실수로 citedby_paper_num를 지웠을 때 복구한 코드
for key in rw.paper_list.keys():
    paper = rw.paper_list[key]
    
    try:
        rw.paper_list[key]['citedby_paper_num'] = len(paper['citedby_paper_list'])
        print(len(paper['citedby_paper_list']), paper['citedby_paper_num'], paper['info']['num_citations'])
        if paper['citedby_paper_num'] > paper['info']['num_citations']:
            print(key, '-> to be modified', len(paper['citedby_paper_list'][paper['info']['num_citations']:]))
            for item in paper['citedby_paper_list'][paper['info']['num_citations']:]:
                print(item)
                paper['citedby_paper_list'].remove(item)
    except KeyError as e:
        print('no citation')

2 2 2
2 2 2
no citation
1 1 1
16 16 16
no citation
1 1 1
no citation
5 5 5
2 2 2
3 3 3
10 10 10
2 2 2
7 7 7
19 19 19
4 4 4
1 1 1
1 1 1
31 31 31
10 10 10
34 34 34
17 17 17
14 14 14
16 16 16
2 2 2
12 12 12
23 23 23
23 23 23
131 131 131
23 23 23
76 76 76
31 31 31
39 39 39
8 8 8
no citation
39 39 39
99 99 99
8 8 8
85 85 85
57 57 57
273 273 273
245 245 245
198 198 198
152 152 152
112 112 112
202 202 202
67 67 67
685 685 685
88 88 88
176 176 176
278 278 278
378 378 378
76 76 76
832 832 832
365 365 365
210 210 981
0 0 200


In [25]:
rw.is_updated = True
rw.update_pkl()

update pkl


In [52]:
# 지우는 코드
for key in rw.paper_list.keys():
    paper = rw.paper_list[key]
    print(paper['info']['num_citations'])
    #print(paper['citedby_paper_list'])
    try:
        del paper['citedby_paper_list']
    except:
        continue
    
rw.is_updated = True
rw.update_pkl()

2
2
0
1
16
0
1
0
5
2
3
10
2
7
19
4
1
1
31
10
34
17
14
16
2
12
23
23
131
23
76
31
39
8
526
39
99
8
85
57
273
245
198
152
112
202
67
685
88
176
278
378
76
832
365
981
200


### 어떤 논문에서 인용이 됐는지 체크하기

In [19]:
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [20]:
a = "Behavior Transformers: Cloning  modes with one stone"
b = "Behavior Transformers: Cloning k modes with one stone"
print(similar(a, b))

0.9904761904761905


In [21]:
from itertools import combinations, permutations
import numpy as np

nums = list(range(1, 58))
print(nums)
perm = list(permutations(nums, 2))
#print(combi)

citation_matrix = np.zeros((57, 57))

# for pair in perm:
#     print(pair[0], pair[1])
#     #print(rw.paper_list[pair[0]])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57]


In [22]:
rw = RelatedWork()

load from pkl


In [23]:
len(rw.paper_list.keys())

57

In [24]:
rw.paper_list[1]['title']

'A Task-Agnostic Regularizer for Diverse Subpolicy Discovery in Hierarchical Reinforcement Learning'

In [78]:
rw.paper_list[46]

{'title': 'Relay Policy Learning: Solving Long-Horizon Tasks via Imitation and Reinforcement Learning',
 'year': '2019',
 'info': {'container_type': 'Publication',
  'source': <PublicationSource.PUBLICATION_SEARCH_SNIPPET: 'PUBLICATION_SEARCH_SNIPPET'>,
  'bib': {'title': 'Relay policy learning: Solving long-horizon tasks via imitation and reinforcement learning',
   'author': ['A Gupta', 'V Kumar', 'C Lynch', 'S Levine'],
   'pub_year': '2019',
   'venue': 'arXiv preprint arXiv …',
   'abstract': 'We present relay policy learning, a method for imitation and reinforcement learning that can solve multi-stage, long-horizon robotic tasks. This general and universally-applicable, two-phase approach consists of an imitation learning stage that produces goal-conditioned hierarchical policies, and a reinforcement learning phase that finetunes these policies for task performance. Our method, while not necessarily perfect at imitation learning, is very amenable to further improvement via enviro

In [87]:
def check_citation(paper1, paper2):
    citation_list = paper1['citedby_paper_list'] # paper1을 인용한 논문들
    #if citation_list == []:
    #    print(citation_list)
    for paper in citation_list:
        #print(paper)
        similarity = similar(paper['title'].lower(), paper2['title'].lower()) # paper2가 paper1을 인용했나?
        #print(similarity)
        if similarity > 0.9:
            #print('citation exists with ', similarity)
            return True, similarity
    return False, 0

In [63]:
bayesian = rw.paper_list[20]
robot_learning = rw.paper_list[14]
residual = rw.paper_list[13]
check_citation(bayesian, residual)

{'title': 'Residual skill policies: Learning an adaptable skill-based action space for reinforcement learning for robotics', 'url': 'https://proceedings.mlr.press/v205/rana23a.html'}
1.0
citation exists with  1.0


(True, 1.0)

In [90]:
ref = 0
for pair in perm:
   #pair = (46, 21)
   #print(pair[0], pair[1])
   paper1 = rw.paper_list[pair[0]]
   paper2 = rw.paper_list[pair[1]]
    
   if pair[0] != ref:
      print("------------------------------------------------------")
      print("{} ({}) is cited by".format(paper1['title'], paper1['year']))
      ref = pair[0]  
   # for paper in paper1['citedby_paper_list']:
   #    if "demonstration" in paper['title'].lower():
   #       print(paper)
   # print(paper2['title'])
   
   
   exists, similarity = check_citation(paper1, paper2)
   if exists:
      print("* {} ({}) with similarity {}".format(paper2['title'], paper2['year'], similarity))
      citation_matrix[pair[0]-1, pair[1]-1] = similarity
   #break

------------------------------------------------------
A Task-Agnostic Regularizer for Diverse Subpolicy Discovery in Hierarchical Reinforcement Learning (2023) is cited by
------------------------------------------------------
Hierarchical Kickstarting for Skill Transfer in Reinforcement Learning (2023) is cited by
------------------------------------------------------
Multi-task Hierarchical Adversarial Inverse Reinforcement Learning (2023) is cited by
------------------------------------------------------
Accelerating Reinforcement Learning for Autonomous Driving using Task-Agnostic and Ego-Centric Motion Skills (2022) is cited by
------------------------------------------------------
Advanced Skills through Multiple Adversarial Motion Priors in Reinforcement Learning (2022) is cited by
------------------------------------------------------
ASPiRe: Adaptive Skill Priors for Reinforcement Learning (2022) is cited by
------------------------------------------------------
Cascaded Comp

In [15]:
for key in rw.paper_list.keys():
    paper = rw.paper_list[key]
    print(key, paper['title'])

1 A Task-Agnostic Regularizer for Diverse Subpolicy Discovery in Hierarchical Reinforcement Learning
2 Hierarchical Kickstarting for Skill Transfer in Reinforcement Learning
3 Multi-task Hierarchical Adversarial Inverse Reinforcement Learning
4 Accelerating Reinforcement Learning for Autonomous Driving using Task-Agnostic and Ego-Centric Motion Skills
5 Advanced Skills through Multiple Adversarial Motion Priors in Reinforcement Learning
6 ASPiRe: Adaptive Skill Priors for Reinforcement Learning
7 Cascaded Compositional Residual Learning for Complex Interactive Behaviors
8 CEIP: Combining Explicit and Implicit Priors for Reinforcement Learning with Demonstrations
9 MPR-RL: Multi-Prior Regularized Reinforcement Learning for Knowledge Transfer
10 Planning to Practice: Efficient Online Fine-Tuning by Composing Goals in Latent Space
11 Priors, Hierarchy, and Information Asymmetry for Skill Transfer in Reinforcement Learning
12 Relay Hindsight Experience Replay: Self-Guided Continual Reinfor

In [51]:
rw.paper_list[20]['citedby_paper_list']

[{'title': 'Residual skill policies: Learning an adaptable skill-based action space for reinforcement learning for robotics',
  'url': 'https://proceedings.mlr.press/v205/rana23a.html'},
 {'title': 'Robot learning of mobile manipulation with reachability behavior priors',
  'url': 'https://ieeexplore.ieee.org/abstract/document/9813580/'},
 {'title': 'Energy optimization of wind turbines via a neural control policy based on reinforcement learning Markov chain Monte Carlo algorithm',
  'url': 'https://www.sciencedirect.com/science/article/pii/S0306261923004725'},
 {'title': 'SkillS: Adaptive Skill Sequencing for Efficient Temporally-Extended Exploration',
  'url': 'https://arxiv.org/abs/2211.13743'},
 {'title': 'Verification of safety critical control policies using kernel methods',
  'url': 'https://ieeexplore.ieee.org/abstract/document/9838224/'},
 {'title': 'Renaissance Robot: Optimal Transport Policy Fusion for Learning Diverse Skills',
  'url': 'https://ieeexplore.ieee.org/abstract/

In [91]:
for key in rw.paper_list.keys():
    paper = rw.paper_list[key]
    if citation_matrix[key-1].sum() > 0:
        print("'{}' is cited by".format(paper['title']))
        b = np.array(citation_matrix[key-1])
        print(b)
        citing_key = np.where(b>0)[0] + 1
        print(citing_key)
        for k in citing_key:
            print(rw.paper_list[k]['year'], rw.paper_list[k]['title'])
        print('\n')

'Skill-based Meta-Reinforcement Learning' is cited by
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[8]
2022 CEIP: Combining Explicit and Implicit Priors for Reinforcement Learning with Demonstrations


'Accelerating Robotic Reinforcement Learning via Parameterized Action Primitives' is cited by
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[14 16 26]
2022 Robot Learning of Mobile Manipulation With Reachability Behavior Priors
2022 Skill-based Model-based Reinforcement Learning
2021 Learning Transferable Motor Skills with Hierarchical Latent Mixture Policies


'Bayesian controller fusion: Leveraging control priors in deep reinforcement learning for robotics' is cited by
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0.